# AWS Networking Fundamentals: Virtual Private Clouds

Amazon Virtual Private Cloud (VPC) is the foundational component for networking in AWS. It enables you to provision a logically isolated section of the AWS Cloud where you can launch AWS resources in a virtual network that you define. A VPC closely resembles a traditional on-premises network with the added benefit of leveraging AWS’s scalable infrastructure.

Just as Azure relies on VNets to create private address spaces, AWS uses VPCs to define and control networking environments. This includes defining IP address ranges, organizing resources into subnets, and controlling routing and connectivity.

In this notebook, we cover the essential concepts and practical setup of an AWS VPC from scratch.

This notebook focuses on:
- Creating a custom VPC
- Defining address space with CIDR
- Creating public and private subnets
- Associating route tables and deploying an internet gateway

The goal is to establish a clean and minimal VPC network suitable as a foundation for more advanced networking configurations.

## VPC: Virtual Private Cloud Basics

A VPC is a regional resource that spans all Availability Zones (AZs) within a region. When you create a VPC, you define an IPv4 CIDR block (e.g., `10.0.0.0/16`) that serves as the address space for all resources within the VPC. Each subnet created within the VPC must fall within this range.

Subnets enable you to divide your network logically. AWS recommends placing public-facing resources like web servers in **public subnets** (those with a route to the internet), and backend services like databases in **private subnets**.

A minimal custom VPC setup includes:
- A VPC with a `/16` CIDR block
- At least one public and one private subnet
- A route table for each subnet
- An Internet Gateway attached to the VPC
- Route table association for internet access

The final VPC will look something like this:

<img src="assets/pics/000_vpc_basics/public_and_privat_subnets_in_vpc.png" alt="Alt text" width="600rem"/>


## Setup: Connecting you to AWS Cloud

First, let's make sure that you can reach your cloud account.

In [1]:
%pip install boto3 python-dotenv

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   -- ------------------------------------- 0.8/13.4 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/13.4 MB 4.8 MB/s eta 0:00:03
   -------- ------------------------------- 2.9/13.4 MB 4.8 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/13.4 MB 4.7 MB/s eta 0:00:03
   ------------ --------------------------- 4.2/13.4 MB 4.3 MB/s eta 0:00:03
   --------------- ------------------------ 5.2/13.4 MB 4.2 MB/s eta 0:00:02
   ----------------- ---------------------- 6.0/13.4 MB 4.1 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/13.4 MB 4.3 MB/s eta 0:00:02
   ------------------------ --------------- 8.1/13.4 MB 4.3 MB/s eta 0:00:02
   -------------------------- ------------- 8.9/13.4 MB 4.4 MB/s eta 0:00:02
   ------------------------------ --------- 10.2/13.4 MB 4.5 MB/s eta 0:00:01
   --------------

In [ ]:
import os
import boto3
from dotenv import load_dotenv

load_dotenv()

In [ ]:
aws_region = os.getenv("AWS_REGION", "eu-north-1")

In [ ]:
session = boto3.Session(region_name=aws_region)

# Create a client for EC2 (which includes VPC and subnet operations)
ec2 = session.client('ec2')

# List all VPCs in the account
vpcs = ec2.describe_vpcs()
vpcs['Vpcs']